In [58]:
#https://blog.naver.com/dev-dog
#https://github.com/Kim-Yeryoung/Midterm_Team3/blob/main/practice/User_defined_modules.ipynb

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
#파일 넣기기
input_file = "한국_기업문화_HR_데이터셋_샘플.csv"
#1. 파일 확인: 
df = pd.read_csv(input_file)
df['이직여부'].value_counts()

이직여부
No     842
Yes    158
Name: count, dtype: int64

In [59]:
'''
2. 데이터 전처리 (15점)

결측치 처리 여부를 확인하고 필요한 전처리를 수행하시오. 

이직여부를 이진값(0, 1)으로 변환하시오.

나머지 범주형 피처는 인코딩하시오.
'''
missing=(df.isnull().sum()/ len(df)*100).sort_values(ascending=False)# 결측치 없음음
missing


df['이직여부'] = df['이직여부'].map({'Yes': 1, 'No': 0})
print(df['이직여부'].unique())
#target label set


df['야근여부'] = df['야근여부'].map({'Yes': 1, 'No': 0})
print(df['야근여부'].unique())


# 변수 분류 (OneHot & gender or Label)
label_encode_cols = []
onehot_cols = []

for col in df.columns:
    if df[col].dtype == 'object' and col not in ['Gender', 'gender', 'Sex', 'sex']:
        n_unique = df[col].nunique()
        if n_unique > 2:
            label_encode_cols.append(col)
        elif n_unique == 2:
            onehot_cols.append(col)

# 범주형 변수 인코딩 함수 (LabelEncoder 사용, 실패 시 해시 기반)
def encode_categoricals(df):
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    cat_cols = [col for col in cat_cols if col != 'year']
    for col in cat_cols:
        try:
            df[col] = LabelEncoder().fit_transform(df[col])
        except:
            df[col] = df[col].astype(str).apply(lambda x: hash(x) % 1000)  # 해시값으로 대체
    return df

label_encode_cols
df[label_encode_cols]=encode_categoricals(df[label_encode_cols]) # Label

print(df['이직여부'].value_counts()) #one-hot은 차원 커지니깐 보류 했음.
print(df['야근여부'].value_counts()) #one-hot은 차원 커지니깐 보류 했음.



[0 1]
[0 1]
이직여부
0    842
1    158
Name: count, dtype: int64
야근여부
0    726
1    274
Name: count, dtype: int64


C:\Users\kimye\AppData\Local\Temp\ipykernel_11152\4271009780.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = LabelEncoder().fit_transform(df[col])
C:\Users\kimye\AppData\Local\Temp\ipykernel_11152\4271009780.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = LabelEncoder().fit_transform(df[col])
C:\Users\kimye\AppData\Local\Temp\ipykernel_11152\4271009780.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [60]:
'''
3. 
피처 선택 (15점)

이직 여부 예측에 유의미하다고 생각되는 피처 5~10개을 선택하고, 선택 근거를 간략하게 기술하시오.

피처 선정에 관한 자세한 기술은 이후에 다룰 예정.(현재는 예측력보다 이해/해석 중점)
feature_target=['이직여부', '업무만족도', '부서', '근무환경만족도', '직무', '연봉인상률', '총경력',
       '야근여부', '성과등급', '워라밸', '현회사근속년수']
'''

df.columns

#업무 만족도, '워라밸, '근무환경만족도': 높아야 이직 안함
#'연봉인상률': 낮으면 이직 하고 돈 안 들어오니 나가고 싶음
#'야근여부': 많으면 나가고 싶음.

feature_target=['이직여부', '업무만족도', '워라밸', '연봉인상률', '총경력','야근여부','근무환경만족도']
selected_df=df[feature_target] # 전처리 된 듯!
df['이직여부'].unique()



array([0, 1])

In [ ]:
'''
5. 모델 훈련 (20점)

학습/검증 데이터를 8:2 비율로 분할하시오.
LogisticRegression, RandomForestClassifier 중 하나를 선택하여 학습하시오.
'''


y = selected_df['이직여부']

X = selected_df.drop('이직여부', axis=1)

# 학습 데이터와 테스트 데이터 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=42)


#의사결정나무 선택: 다양한 요인이 복합적으로 작용하는 이직률 예측할 수 있게 해주니깐.
from sklearn.tree import DecisionTreeClassifier

model=DecisionTreeClassifier(criterion = 'entropy', max_depth=3, min_samples_split=5)
model.fit(X_train, y_train)

print('Score: ',model.score(X_train, y_train))
print('Score: ',model.score(X_test, y_test))


Score:  0.86125
Score:  0.86


In [ ]:
'''6. 성능 검증 (10점)

테스트셋에서 정확도, 혼동행렬을 계산하시오. 결과에 대해 간단히 해석하시오.'''

#: 모델 예측 성능 평가
from sklearn.metrics import accuracy_score

y_pred=model.predict(X_test)
print('Test Accuracy is', accuracy_score(y_test, y_pred)*100)
#해석: 이직 하는 사람을 예측하는 모델 만들어서 학습 데이터로 학습 시키고 테스트 데이터로 테스트로 실제와 학습 모델이 예측한 것에 대해서 어느정도 차이나는지 평가
#테스트 데이터에 대한 정확도 출력 (예: 86%) : 100명 중 86명 정도 이직 여부 맞추는 모델 만든 것이다. 
# -->정확도 자체는 파악할 수 있으나 정확히 어떤 부분에서 오답이 발새하는지 알 수 없음음-->confusion matrix로!

#Accuracy=(올바르게 예측한 샘플 개수)/(전체 샘플 개수}=(TP+TN)/(TP+TN+FP+FN)


Test Accuracy is 86.0


In [ ]:
# 혼동 행렬(Confusion Matrix) 확인
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)
#좀 더 자세하게 TP: 160 명 맞췄고 TN: 12명, FP: 23, FN:12명 정도로 파악 가능


array([[160,   5],
       [ 23,  12]])